In [1]:
!pip install transformers datasets sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
!python --version

Python 3.9.16


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tensorflow as tf
from transformers import AutoTokenizer, AdamWeightDecay, TFAutoModelForSeq2SeqLM
from transformers.keras_callbacks import KerasMetricCallback
from transformers.keras_callbacks import PushToHubCallback
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu
from datasets import load_dataset
# import evaluate
from transformers import GPT2Tokenizer, GPT2Model
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import AutoModelWithLMHead, AutoTokenizer
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch 
from tensorflow.keras.preprocessing.text import Tokenizer
import string
device = torch.cuda

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/main_dataset_with_space_emojis.csv')

In [5]:
emoji = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/dataset_with_emojis_in_1_column.csv')

In [6]:
df = emoji

In [7]:
df.dropna(inplace = True, how = 'any')

In [8]:
df

,Unnamed: 0,0,1
0,0,"""Falling for the fairytale",🌹 🥀
1,1,"""A tale as old as time ️",🕰 💕
2,2,"""Happily ever after with my beast",👸 🦁
3,3,"""Capturing a moment in time",📸 🎬
4,4,"""Inked memories for a lifetime ️️",🖋 🎞
...,...,...,...
21910,21910,A touch of gold can make all the difference.,🌟
21911,21911,Frame your world in glittering gold. ️,✨ 🖼
21912,21912,"Hungry, hungry hippos enjoying some fresh gree...",🦛 🌿
21913,21913,A hippo's gotta eat! Chowin' down on some leaf...,🍃 🦛


In [9]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [10]:
train, test = train_test_split(df, test_size=0.1)

In [11]:
train, val = train_test_split(train, test_size = 0.1)

In [ ]:
train

,Original Caption,Creative Caption 1,Creative Caption 2,Creative Caption 3
2289,Actors attend the world premiere,"✨ Glitz, glamour, and the silver screen ✨",It's showtime 🎬,The stars have arrived 🌟
3646,"""Person hugs her son as supporters walk past a...","""Emotional embrace - a heartfelt hug between a...","""Family moment - a touching scene of a person ...","""Supporter's love - a mother hugging her son a..."
37,Actor at the premiere of the new movie,The star of the show has arrived 🌟 🎬 #MoviePr...,Red carpet ready 👠 👞 #CelebrityStyle #Fashion,"Lights, camera, action 📽 ️🎞 ️ #Hollywood #Movi..."
231,players surround football player in celebratio...,That feeling when your team scores a goal! 🙌 ⚽...,Nothing beats the thrill of victory and the ru...,Our team knows how to score a goal and celebra...
3718,vector illustration of the beautiful girl with...,Beauty that goes beyond the surface. #BeautyWi...,Art that evokes emotion and inspiration. #ArtI...,A glimpse into the artist's mind. #ArtisticVis...
...,...,...,...,...
938,person and groom are taking a moment to themse...,Happily ever after begins today! ❤ ️👰 🤵 #Wedd...,Two hearts become one on this special day ❤ ️ ...,Together is a beautiful place to be ❤ ️ #Forev...
659,but now you have to carry umbrellas at the sam...,"When life gives you rain, dance in the puddles...",Don't let a little rain dampen your spirit 🌦 ️...,Umbrellas up! It's raining good vibes today 🌧 ...
1101,Accommodation type for rent in an apartment - ...,🏠 🔑 🛋 ️ Your home away from home! Rent this co...,🌇 🏢 👩 ‍💼 Live the city life in style! Rent th...,🌴 🏝 ️🌊 Live the island life! Rent this beachf...
4551,Long country road with white lines down the ce...,"""Endless horizon on a serene country road! 🛣 ️...","""Tranquil country road leading to the distant ...","""Picturesque country road with a solitary tree..."


In [ ]:
train1 = train.loc[:, ['Original Caption', 'Creative Caption 1']].reset_index(drop = True)
train2 = train.loc[:, ['Original Caption', 'Creative Caption 2']].reset_index(drop = True)
train3 = train.loc[:, ['Original Caption', 'Creative Caption 3']].reset_index(drop = True)

In [ ]:
train1.rename(columns = {'Creative Caption 1': 'Creative Caption'}, inplace = True)
train2.rename(columns = {'Creative Caption 2': 'Creative Caption'}, inplace = True)
train3.rename(columns = {'Creative Caption 3': 'Creative Caption'}, inplace = True)

In [ ]:
train = pd.concat([train1, train2, train3], ignore_index = True)

In [ ]:
val1 = val.loc[:, ['Original Caption', 'Creative Caption 1']].reset_index(drop = True)
val2 = val.loc[:, ['Original Caption', 'Creative Caption 2']].reset_index(drop = True)
val3 = val.loc[:, ['Original Caption', 'Creative Caption 3']].reset_index(drop = True)

val1.rename(columns = {'Creative Caption 1': 'Creative Caption'}, inplace = True)
val2.rename(columns = {'Creative Caption 2': 'Creative Caption'}, inplace = True)
val3.rename(columns = {'Creative Caption 3': 'Creative Caption'}, inplace = True)

val = pd.concat([val1, val2, val3], ignore_index = True)

In [ ]:
test1 = test.loc[:, ['Original Caption', 'Creative Caption 1']].reset_index(drop = True)
test2 = test.loc[:, ['Original Caption', 'Creative Caption 2']].reset_index(drop = True)
test3 = test.loc[:, ['Original Caption', 'Creative Caption 3']].reset_index(drop = True)

test1.rename(columns = {'Creative Caption 1': 'Creative Caption'}, inplace = True)
test2.rename(columns = {'Creative Caption 2': 'Creative Caption'}, inplace = True)
test3.rename(columns = {'Creative Caption 3': 'Creative Caption'}, inplace = True)

test = pd.concat([test1, test2, test3], ignore_index = True)

In [ ]:
train.columns

Index(['0', '1'], dtype='object')

In [12]:
train.rename(columns = {'0': 'caption', '1': 'emoji'}, inplace = True)
val.rename(columns = {'0': 'caption', '1': 'emoji'}, inplace = True)
test.rename(columns = {'0': 'caption', '1': 'emoji'}, inplace = True)

In [13]:
train.loc[:, ['caption']] = train.loc[:, ['caption']].applymap(lambda x: '<SOS> ' + x + ' <EOS>' if type(x) == str else x)
val.loc[:, ['caption']] = val.loc[:, ['caption']].applymap(lambda x: '<SOS> ' + x + ' <EOS>' if type(x) == str else x)
test.loc[:, ['caption']] = test.loc[:, ['caption']].applymap(lambda x: '<SOS> ' + x + ' <EOS>' if type(x) == str else x)

In [14]:
train = Dataset.from_pandas(train.reset_index(drop = True))
val = Dataset.from_pandas(val.reset_index(drop = True))
test = Dataset.from_pandas(test.reset_index(drop = True))

In [15]:
max_len = 0
for pair in train:
  max_len = max(max_len, len(pair['caption'].split()))

for pair in val:
  max_len = max(max_len, len(pair['caption'].split()))

for pair in test:
  max_len = max(max_len, len(pair['caption'].split()))

max_len

# max_len = 0
# for pair in train:
#   max_len = max(max_len, len(pair['Original Caption'].split()), len(pair['Creative Caption'].split()))

# for pair in val:
#   max_len = max(max_len, len(pair['Original Caption'].split()), len(pair['Creative Caption'].split()))

# for pair in test:
#   max_len = max(max_len, len(pair['Original Caption'].split()), len(pair['Creative Caption'].split()))

# max_len

39

In [16]:
emoji_dataset = Dataset.from_pandas(emoji)

In [ ]:
emoji_dataset

Dataset({
    features: ['Unnamed: 0', '0', '1'],
    num_rows: 31138
})

In [17]:
self_tokenizer = Tokenizer(filters='', oov_token='<OOV>')
self_tokenizer.fit_on_texts(train['emoji'])
# self_tokenizer.fit_on_texts(train['Creative Caption'])
# self_tokenizer.fit_on_texts(val['emoji'])
# # self_tokenizer.fit_on_texts(val['Creative Caption'])
# self_tokenizer.fit_on_texts(test['emoji'])
# self_tokenizer.fit_on_texts(test['Creative Caption'])

In [18]:
checkpoint = 'mrm8488/t5-base-finetuned-common_gen'
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-common_gen")

In [19]:
new_tokens = set(self_tokenizer.word_index.keys()) - set(tokenizer.vocab.keys())

In [20]:
len(new_tokens)

926

In [ ]:
new_tokens_clean = set()
for token in new_tokens:
  new_tokens_clean.add(token.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
temp = set()
new_tokens = set()
for tokens in new_tokens_clean:
  if '<SOS>'  in tokens or '<EOS>' in tokens:
    continue
  if tokens.isalnum() == False:
    temp.add(tokens)
  else:
    new_tokens.add(tokens)

In [ ]:
new_tokens = []

for emoji in list(temp):
  new_tokens.append(emoji)

In [21]:
tokenizer.add_tokens(list(new_tokens))

926

In [22]:
special_tokens = ['<SOS>', '<EOS>', 'add emojis: ']
tokenizer.add_tokens(special_tokens, special_tokens=True)

3

In [23]:
prefix = "add emojis: "

def preprocess_function(examples):
  # print(examples)
  inputs = [prefix + example for example in examples['caption']]
  targets = [example for example in examples['emoji']]
  # print(inputs)
  # print(targets)
  model_inputs = tokenizer(inputs, max_length = max_len, truncation = True, padding = True, text_target = targets, return_tensors = 'pt')
  # print(type(model_inputs))
  return model_inputs

In [24]:
train = train.map(preprocess_function, batched = True)
val = val.map(preprocess_function, batched = True)

Map:   0%|          | 0/16279 [00:00<?, ? examples/s]

Map:   0%|          | 0/1809 [00:00<?, ? examples/s]

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
# metric = evaluate.load('sacrebleu')
# rouge_metric = load_metric('rouge')

# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [[label.strip()] for label in labels]

#     return preds, labels


# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {"bleu": result["score"]}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     result = {k: round(v, 4) for k, v in result.items()}
#     return result

In [26]:
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-common_gen").cuda()
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


Embedding(33029, 768)

In [27]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Information Retrieval Project/saved_models/common_gen_emoji_final",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset = val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [28]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.058378
2,1.393400,1.002747
3,1.393400,0.982097
4,0.980200,0.968004
5,0.980200,0.945795
6,0.930000,0.926633
7,0.930000,0.921028
8,0.892500,0.923035
9,0.892500,0.919851
10,0.868300,0.921783


TrainOutput(global_step=2550, training_loss=1.009942658368279, metrics={'train_runtime': 1079.923, 'train_samples_per_second': 150.742, 'train_steps_per_second': 2.361, 'total_flos': 7551089736652800.0, 'train_loss': 1.009942658368279, 'epoch': 10.0})

In [29]:
trainer.save_model('/content/drive/MyDrive/Information Retrieval Project/saved_models/common_gen_emoji_final_model')

In [ ]:
model = AutoModelWithLMHead.from_pretrained('/content/drive/MyDrive/Information Retrieval Project/saved_models/common_gen_7k_model_emoji').cuda()

/usr/local/lib/python3.9/dist-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
temp = prefix + test['caption'][1000]
# temp = prefix + 'man with his dog on beach playing'
temp = tokenizer(temp, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt').input_ids.cuda()
temp = model.generate(temp)

In [ ]:
temp

tensor([[    0, 32494,     3,     1]], device='cuda:0')

In [ ]:
tokenizer.batch_decode(temp)

['<pad>⚽ </s>']

In [ ]:
test['caption'][1000]

'<SOS> A hard-earned win   <EOS>'

In [ ]:
test["Creative Caption"][0]

KeyError: ignored

In [ ]:
def testing(examples):
  inputs = [prefix + example for example in examples['Original Caption']]
  model_inputs = tokenizer(inputs, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt')
  return model_inputs

In [ ]:
test_dataset = test.map(testing, batched = True)

Map:   0%|          | 0/879 [00:00<?, ? examples/s]

In [ ]:
test_dataset

Dataset({
    features: ['Original Caption', 'Creative Caption', 'input_ids', 'attention_mask'],
    num_rows: 879
})

In [ ]:
model_output_ids = []
for sample in test:
  temp = prefix + sample['Original Caption']
  temp = tokenizer(temp, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt').input_ids.cuda()
  temp = model.generate(temp)
  model_output_ids.append(temp)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
model_outputs = []

for ids in model_output_ids:
  model_outputs.append(tokenizer.batch_decode(ids))

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

In [ ]:
bleu_original = 0
bleu_insta = 0

for i in range(len(model_outputs)):
  # print(test[i]['Original Caption'], ' space ',model_outputs[i])
  bleu_original += sentence_bleu([test[i]['Original Caption']], model_outputs[i][0])
  bleu_insta += sentence_bleu([test[i]['Creative Caption']], model_outputs[i][0])

bleu_original /= len(model_outputs)
bleu_insta /= len(model_outputs)

In [ ]:
bleu_insta, bleu_original

(0.27613493758097335, 0.3874061470404573)

In [ ]:
import pickle

In [ ]:
pickle.dump(train, open('/content/drive/MyDrive/Information Retrieval Project/common_gen_7k_files/train_set.pkl', 'wb'))

In [ ]:
pickle.dump(val, open('/content/drive/MyDrive/Information Retrieval Project/common_gen_7k_files/val_set.pkl', 'wb'))

In [ ]:
pickle.dump(val, open('/content/drive/MyDrive/Information Retrieval Project/common_gen_7k_files/test_set.pkl', 'wb'))

In [ ]:
pickle.dump(model_outputs, open('/content/drive/MyDrive/Information Retrieval Project/common_gen_7k_files/model_outputs.pkl', 'wb'))

In [ ]:
conceptual_captions = pickle.load(open('/content/drive/MyDrive/Information Retrieval Project/vit-gpt2/conceptual_results/data_with_generated_captions.pkl', 'rb'))

In [ ]:
conceptual_captions

,Original Caption,Image,Generated_descriptive_caption
1000,sparkling glittery gold uppercase or capital l...,https://thumb1.shutterstock.com/display_pic_wi...,a white and black vase with a bird on it
1001,actor arrives for the premiere,https://media.gettyimages.com/photos/kristen-b...,a woman in a red dress standing in front of a ...
1002,i jumped out of a helicopter and you can see t...,http://i.imgur.com/ynzWXIO.jpg,a person holding a surfboard in their hand
1003,athlete prepares to score a touchdown as his t...,http://photos.mycapture.com/THST/1561986/44629...,#
1004,baseball player shakes hands with baseball pla...,https://billieweiss.files.wordpress.com/2014/0...,#
...,...,...,...
1995,a bit of the view from our table on new year '...,https://media-cdn.tripadvisor.com/media/photo-...,a city with lots of buildings and a river
1996,actor wears a slim grey suit to awards .,http://www.windsorstar.com/cms/binary/6493391....,#
1997,a side street viewed not .,https://www.jefflindsay.com/gif/asia/shanghai_...,"a sign that says ""don't drink in the city"""
1998,map with long shadow and white outline on blue...,https://thumb1.shutterstock.com/display_pic_wi...,#


In [ ]:
con_cap = conceptual_captions[conceptual_captions['Generated_descriptive_caption'] != '#']

In [ ]:
con_dataset = Dataset.from_pandas(con_cap)

In [ ]:
con_output_ids = []

for sample in con_dataset:
  temp = prefix + sample['Generated_descriptive_caption']
  temp = tokenizer(temp, max_length = max_len, truncation = True, padding = True, return_tensors = 'pt').input_ids.cuda()
  temp = model.generate(temp)
  con_output_ids.append(temp)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
con_outputs = []

for ids in con_output_ids:
  con_outputs.append(tokenizer.batch_decode(ids))

In [ ]:
con_bleu_original = 0
con_bleu_desc = 0
for i in range(len(con_outputs)):
  # print(con_dataset[i]['Original Caption'], ' space ',model_outputs[i])
  con_bleu_original += sentence_bleu([con_dataset[i]['Original Caption']], con_outputs[i][0])
  con_bleu_desc += sentence_bleu([con_dataset[i]['Generated_descriptive_caption']], con_outputs[i][0])

con_bleu_original /= len(con_outputs)
con_bleu_desc /= len(con_outputs)

/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

In [ ]:
con_bleu_original, con_bleu_desc

(0.10281281645921732, 0.3433259959521282)

In [ ]:
from PIL import Image
import requests
from io import BytesIO

In [ ]:
for i in range(5):
  response = requests.get(con_dataset[i]['Image'])
  img = Image.open(BytesIO(response.content))
  img.show()
  print(con_outputs[i])
  print(con_dataset[i]['Original Caption'])
  print(con_dataset[i]['Generated_descriptive_caption'])

In [ ]:
pickle.dump(con_dataset, open('/content/drive/MyDrive/Information Retrieval Project/common_gen_7k_files/conceptual_dataset.pkl', 'wb'))
pickle.dump(con_outputs, open('/content/drive/MyDrive/Information Retrieval Project/common_gen_7k_files/desc_con_outputs.pkl', 'wb'))

In [ ]:
emoji_dataset = pd.read_csv('/content/drive/MyDrive/Information Retrieval Project/Instagram worthy captions dataset/final_data_with_emojis.csv')

In [ ]:
emoji_dataset

,Unnamed: 0,0,1
0,0,"""Falling for the fairytale",🌹
1,1,"""Falling for the fairytale",🥀
2,2,"""A tale as old as time ️",🕰
3,3,"""A tale as old as time ️",💕
4,4,"""Happily ever after with my beast",👸
...,...,...,...
31133,31133,Charting my course to new adventures. ️,🗺
31134,31134,The world is your playground.,🌏
31135,31135,Captivated by the intricate details of ancient...,🏛
31136,31136,The beauty of art is in the details.,🎨
